In [5]:
# -*- coding: utf-8 -*-
"""
Created on Thu May 12 23:24:54 2016

@author: stuka

"""

import numpy as np
import nltk
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
import os
import io
import re
from collections import defaultdict
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import math
import pandas as pd
import sys
import gensim
from string import punctuation
from scipy.cluster.vq import kmeans,vq

#Cleaning the house
os.getcwd()
os.chdir("/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/data/raw")
os.listdir(".")

path_to_raw = "/home/christian/Documents/ITAM_CDA/Mineria de texto/final_project/mineria-texto-diario-debates/data/"


In [ ]:
#Debe venir de fuera
"""
Christian - pon tu path local en path_to_raw
Mete manualmente archivos a una carpeta_christian y corre este codigo
file_names = [f for f in os.listdir(path_to_raw+'carpeta_christian/') if f.endswith('.txt')]
documentos =[io.open(f,'rt') for f in file_names]


"""

file_names = [f for f in os.listdir(path_to_raw+'raw/') if f.endswith('.txt')]

documentos =[io.open(f,'rt',encoding='ISO-8859-1') for f in file_names]


for file_name in file_names:
    sourceEncoding = "iso-8859-1"
    targetEncoding = "utf-8"
    source = io.open(file_name,'rt',encoding='ISO-8859-1')
    target = open(path_to_raw+'encoded/'+file_name, "w")
    target.write(source.read().encode(targetEncoding))
    
documentos =[io.open(path_to_raw+'encoded/'+f,'rt') for f in file_names]

In [ ]:
#Pruebas
raw = documentos[0].read()

documentos[1]

dir(data[0])
data[0].name
#Codigo para probar stopterm
for documento in documentos:
    print(len(documento.read().replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")))

In [ ]:
    
#Este es el buen codigo
"""
documento trae un file abierto
lo consumo a memoria con read() a un solo string
lo parto segun keyterms = Honorable asamblea
"""
for documento in documentos:
    documento_nombre = documento.name.rsplit('/',1)[1].split('.')[0]
    raw = documento.read()
    for i,tematica in enumerate(raw.replace("Honorable Asamblea:","Honorable asamblea:").split("Honorable asamblea:")):
        with io.open(path_to_raw+'tematicas/'+documento_nombre+'_'+str(i+1)+'.txt', mode="w") as newfile:
            tematica_limpia = mataAcentos(strip_punctuation(tematica.replace('\n',' ')))
            newfile.write(tematica_limpia)

def mataAcentos(s):
    charstosub = pd.DataFrame(zip([u'á', u'é', u'í', u'ó', u'ú',u'"',u'“',u'”',u',',u'\.',u'ñ',u'\!',u'\¡'],[u'a', u'e', u'i', u'o', u'u',u'',u'',u'',u'',u'',u'n',u'',u''])) 
    data = s.lower()
    for row in charstosub.iterrows():
        data = re.sub(row[1][0],row[1][1],data)
    return data
##Este no funciona aun
"""
Remover acentos y puntuacion parece no ser lo mejor


def cleanText(corpus):
    #punctuation = """.,?!:;(){}[]"""
    punctuation = punctuation
    corpus = [z.lower().replace('\n','') for z in corpus]
    corpus = [z.replace('<br />', ' ') for z in corpus]

    #treat punctuation as individual words
    for c in punctuation:
        corpus = [z.replace(c, ' %s '%c) for z in corpus]
    corpus = [z.split() for z in corpus]
    return corpus
"""

#Esto si funciona
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

In [142]:
"""
Christian!!!!!
Crea una carpeta prueba y mete alli documentos de prueba para RAKE
"""
file_names_tematicas = [f for f in os.listdir(path_to_raw+'tematicas/') if f.endswith('.txt')]
sentences = []
i=0
dicto = {}
for doc in file_names_tematicas:
    f = io.open(path_to_raw+'tematicas/'+doc,'rt')
    sentences.append(gensim.models.doc2vec.LabeledSentence(f.read().split(),[doc]))
    f.close()
    
    
#Pruebas
type(sentences[0])
#sentences[0:5]
#print(sentences[0:5])


gensim.models.doc2vec.LabeledSentence

In [143]:
#Modelo
min_count = 5 # Ignora todo documento que tenga cinco palabras o menos
size = 60 # Numero de dimensiones (tanto de las palabras como de los documentos)
window = 10 # Cuantas palabras hacia adelante / atrás usa para trabajar
 
model = gensim.models.doc2vec.Doc2Vec(sentences,size = size, window = window, min_count = min_count)

In [144]:
dir(model)
len(model.vocab)
model.similarity('pri','pan')

0.89159278462155822

In [145]:
def checa(word):
    for i in model.most_similar(word):
        print(i)
        
checa(u'fraudes')

(u'procesos', 0.8570495843887329)
(u'deseos', 0.7876545190811157)
(u'padrones', 0.7757041454315186)
(u'aumentos', 0.768700122833252)
(u'argumentos', 0.7674710750579834)
(u'comprobantes', 0.7639429569244385)
(u'nombres', 0.763940691947937)
(u'escrutinios', 0.7610893249511719)
(u'triunfos', 0.7582401037216187)
(u'razonamientos', 0.7577353119850159)


In [147]:
#print model.docvecs.most_similar(["SENT_1"])
#model.docvecs['SENT_37']
#dir(model.docvecs)
#model.docvecs.doctags
#dir(model)

### K means

In [148]:
mylist = []
tags=[]
for doc in model.docvecs.doctags:
    tags.append(doc)    
    mylist.append(model.docvecs[doc])



mat = np.array(mylist)

In [149]:
for i in range(10):
    agrupamiento = KMeans(n_clusters = i+1)
    agrupamiento.fit(mat)
#    mylist['clasificacion_'] = agrupamiento.labels_.tolist()

In [150]:
from scipy.cluster.vq import kmeans,vq
K = range(1,10)

KM = [kmeans(mat,k) for k in K]
centroids = [cent for (cent,var) in KM] 
avgWithinSS = [var for (cent,var) in KM] # suma de cuadrados promedio intra-cluster 

In [151]:
##### plot ###
kIdx = 4

# elbow curve
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(K, avgWithinSS, 'b*-')
ax.plot(K[kIdx], avgWithinSS[kIdx], marker='o', markersize=12, 
    markeredgewidth=2, markeredgecolor='r', markerfacecolor='None')
plt.grid(True)
plt.xlabel('Numero de clusters')
plt.ylabel('Suma de cuadrados promedio intra-cluster')
plt.title('Criterio del codo para K-Medias')

plt.show()

In [152]:
# KMeans con el resultado del criterio del codo
agrupamiento = KMeans(n_clusters = 5)
agrupamiento.fit(mat)
agrupamiento.labels_

array([3, 0, 3, ..., 0, 0, 4], dtype=int32)

In [153]:
pd.DataFrame(zip(tags, agrupamiento.labels_.tolist()))
#tags[''] = agrupamiento.labels_.tolist()

,0,1
0,dd_1946_293.txt,3
1,dd_1970_328.txt,0
2,dd_1964_504.txt,3
3,dd_1964_142.txt,0
4,dd_1946_172.txt,3
5,dd_1964_59.txt,3
6,dd_1970_432.txt,4
7,dd_1964_323.txt,3
8,dd_1970_202.txt,0
9,dd_1964_697.txt,3


In [ ]:
for i in len(range(10)):
    
X_suma = documentos_suma[['vector1','vector2']].as_matrix()
agrupamiento = KMeans(n_clusters = i)
agrupamiento.fit(X_suma)

In [ ]:
for doc in model.doc